좋은 언어 모델이란 실제 우리가 쓰는 언어와 최대한 비슷하게 확률 분포를 근사하는 모델일 것입니다. 많이 쓰이는 문장이나 표현일수록 확률을 높게 예측해야 하며, 적게 쓰이는 문장이나 표현일 수록 확률을 낮게 예측해야 합니다.

문장의 앞 부분이 주어지고, 다음에 나타나는 단어의 확률 분포가 실제 테스트 문장의 다음 단어에 대해 높은 확률을 갖는다면 더 좋은 언어 모델이라고 할 수 있습니다.

### Perplexity (PPL)

**퍼플렉시티 (PPL)**를 측정하는 방법은 $정량\ 평가^{extrinsic\ evalutation}$ 방법의 하나입니다. PPL은 문장의 길이를 반영하여 확률값을 정규화한 값이라고 할 수 있습니다. PPL을 이용하여 언어 모델에서 테스트 문장들의 점수를 구하고, 이를 기반으로 언어 모델의 성능을 측정합니다.

<br></br>
$$
PPL(w_1, w_2, \dots, w_n) = P(w_1, w_2, \dots, w_n)^{-\frac{1}{n}} \\
= \sqrt[n]{\frac{1}{P(w_1,w_2,\dots,w_n)}}
$$
<br></br>

문장이 길어지면 문장의 확률은 굉장히 작아집니다. 이는 연쇄법칙에 따라 조건부 확률들의 곱으로 바꿔 표현해보면 알 수 있습니다. 따라서 문장의 길이 n으로 제곱근을 취해 기하평균을 구하고, 문장 길이에 대해 정규화를 수행합니다. 문장의 확률값이 분모에 들어가므로, 확률값이 높을수록 PPL이 작아집니다.

테스트 문장에 대해 확률을 높게 예측할수록 좋은 언어 모델이기 때문에, PPL이 작을수록 좋은 언어 모델입니다. 보통 n-gram의 n이 클수록 더 낮은 PPL을 보여줍니다.

앞에서 본 PPL 수식을 연쇄법칙에 의해 다음과 같이 표현합니다.

<br></br>
$$
= \sqrt[n]{\frac{1}{\prod_{i=1}^n P(w_i|w_1,\dots,w_{i-1})}}
$$
<br></br>

여기에 n-gram이 적용될 경우 다음과 같이 표현할 수 있습니다.

<br></br>
$$
\approx \sqrt[n]{\frac{1}{\prod_{i=1}^n P(w_i|w_{i-n+1},\dots,w_{i-1})}}
$$
<br></br>

### PPL의 해석

주사위를 던져서 나오는 값을 통해 수열을 만들어낸다고 해보겠습니다. 1부터 6까지 숫자의 출현 확률은 균등 분포를 따르므로 모두 같다고 가정합니다. 이때 N번 주사위를 던져 얻어내는 수열에 대한 PPL은 다음과 같습니다.

<br></br>
$$
PPL(x) = [(\frac{1}{6})^N]^{-\frac{1}{N}} = 6
$$
<br></br>

매 time-step마다 가능한 경우의 수인 6이 PPL로 나왔습니다. 즉, PPL은 우리가 뻗어나갈 수 있는 가지^{branch}의 숫자를 의미하기도 합니다.

다른 예를 들어 만약 20,000개의 어휘로 이루어진 뉴스 기사에 대해 PPL을 측정할때, 단어의 출현 확률이 모두 같다면 PPL은 20,000이 될 것입니다. 하지만 만약 3-gram을 사용한 언어 모델을 만들어 측정한 PPL값이 30이 나왔다면, 해당 신문기사에서 이 언어 모델을 기반으로 매번 기사의 앞부분을 통해 다음 단어를 예측할 때마다 평균적으로 30개의 후보 단어중에 선택할 수 있다는 이야기가 됩니다. 이처럼 PPL은 단순히 언어 모델의 성능 측정뿐만 아니라 그 값의 의미도 가능해볼 수 있습니다.

### 엔트로피와 PPL의 관계

엔트로피는 정보량의 평균을 의미합니다. 정보량이 낮으면 확률 분포는 뾰족한 모양이 되고, 반대로 정보량이 높으면 확률 분포는 납작해집니다. 정보량은 놀람의 정도를 나타내는 수치이기도 합니다. PPL 또한 사전의 정의에 따르면 "곤혹" 또는 "당혹"이라는 뜻입니다.

먼저 실제 언어 모델의 분포 P(x) 또는 출현 가능한 문장들의 집합 W에서 길이 n의 문장 $w_{1:n}$을 샘플링했을때, 우리 언어 모델 분포 $P_\theta(x)$의 엔트로피를 나타내면 다음과 같습니다.

<br></br>
$$
H_n(P,P_\theta) = -\sum_{w_{1:n} \in W} P(w_{1:n})logP(w_{1:n})
$$
<br></br>

여기서 몬테카를로 샘플링을 통해 앞의 수식을 근사할 수 있습니다. 그리고 샘플링 횟수 k가 1일때도 생각해볼 수 있습니다.

<br></br>
$$
H_n(P,P_\theta) = -\sum_{w_{1:n} \in W} P(w_{1:n})logP(w_{1:n}) \\
\approx - \frac{1}{k} \sum_{i=1}^k log P_\theta(w_{1:n}^k) \\
\approx - log P_\theta(w_{1:n}) \\
= - \sum_{i=1}^n log P_\theta(w_i|w_{<i})
$$
<br></br>

앞서와 같이 $H_n(P,P_\theta)$을 근사할 수 있지만, 사실 문장은 시퀀셜 데이터이므로 Entropy Rate라는 개념을 사용하여 다음과 같이 단어당 평균 엔트로피로 나타낼 수 있습니다. 그리고 마찬가지로 몬테카를로 샘플링을 적용할 수도 있습니다.

<br></br>
$$
H_n(P,P_\theta) \approx - \frac{1}{n} \sum_{w_{1:n} \in W} \sum_{i=1}^n P(w_i|w_{<i}) log P_\theta(w_i|w_{<i}) \\
\approx - \frac{1}{n} \sum_{i=1}^n log P(w_i|w_{<i}) \\
= L(w_{1:n})
$$
<br></br>

이 수식을 조금 더 바꾼다면:

<br></br>
$$
L(w_{1:n}) = - \frac{1}{n} \sum_{i=1}^n log P(w_i|w_{<i}) \\
= -\frac{1}{n} log \prod_{i=1}^n P_\theta(w_i|w_{<i}) \\
= log (\prod_{i=1}^n P_\theta(w_i|w_{<i}))^{-\frac{1}{n}} \\
= log \sqrt[n]{\frac{1}{\prod_{i=1}^n P_\theta(w_i|w_{<i})}}
$$
<br></br>

PPL 수식을 다시 떠올려봅시다.

<br></br>
$$
PPL(w_1, w_2, \dots, w_n) = \sqrt[n]{\frac{1}{\prod_{i=1}^n P(w_i|w_1,\dots,w_{i-1})}}
$$
<br></br>

앞서 교차 엔트로피로부터 이끌어냈던 수식과 비슷한 형태임을 알 수 있습니다. 따라서 PPL과 교차 엔트로피의 관계는 다음과 같습니다.

<br></br>
$$
PPL = exp(Cross Entropy)
$$
<br></br>

따라서 MLE를 통해 파라미터 $\theta$를 학습할대 교차 엔트로피를 통해 얻은 손실값 ($P_\theta$의 로그 확률값)에 exp를 취함으로서 PPL을 얻어 언어 모델의 성능을 나타낼 수 있습니다.